In [ ]:
from datasets import load_dataset
import os
import sys
sys.path.append('../../src/')

from mask_dataset import MaskedDataset
dataset = MaskedDataset(model_name="mbert")

In [ ]:
from wiki_evaluation import evaluate_corr_between_wikipages_and_p1
evaluate_corr_between_wikipages_p1(dataset)


In [ ]:
cnt = 0
for uuid in aaa['zh']:
    if len(aaa['zh'][uuid]) > 0:
        cnt += 1

cnt, len(aaa['zh'])

In [ ]:
# from wiki_2018_dump import check_entity_existence_in_titles
lang2obj, lang2objuri = dataset.get_lang2objs()
lang2sub, lang2suburi = dataset.get_lang2subs()
SRC_DATA_ROOT = "/disk/xzhao/datasets/wikipedia_2018_octnov"

def check_entity_existence_in_titles(subjects, lang):
    import numpy as np
    lang_path = os.path.join(SRC_DATA_ROOT, "title", lang)
    assert len(os.listdir(lang_path)) == 1

    # obj_prefix_sets = set([obj[:3] for obj in objects])
    sub_prefix_sets = set([sub[:3] for sub in subjects])

    title_fn = os.path.join(lang_path, os.listdir(lang_path)[0])
    titles = set()
    with open(title_fn, 'r') as fp:
        for idx, line in enumerate(fp):
            line = line.strip()
            if idx == 0 or line == "":
                continue
            try:
                assert len(line.split("\t")) == 2
                title = line.split()[1]
                # if title[:3] in obj_prefix_sets or title[:3] in sub_prefix_sets:
                if title[:3] in sub_prefix_sets:
                    titles.add(title)
            except Exception as e:
                aa = line.split('\t')
                print(f"Assertion error: {aa}")
                
    # obj_exist_label = np.zeros((len(objects), ))
    unmatched_subs = []
    sub_exist_label = np.zeros((len(subjects), ))
    for idx, sub in enumerate(subjects):
        if sub in titles:
            sub_exist_label[idx] = 1
        else:
            unmatched_subs.append(sub)

    # for idx, obj in enumerate(obj_prefix_sets):
    #     if obj in titles:
    #         obj_exist_label[idx] = 1

    # return lang, obj_exist_label, sub_exist_label
    return lang, sub_exist_label, unmatched_subs

lang = 'en'
lang, sub_exist_label, unmatched_subs = check_entity_existence_in_titles(lang2sub[lang], lang)
print(len(sub_exist_label), sum(sub_exist_label))

In [ ]:
def chunk_list(input_list, chunk_size):
    res = []
    for i in range(0, len(input_list), chunk_size):
        res.append(input_list[i:i+chunk_size])
    return res

sub_info = dataset.sub_info
lang2subs = {}
for lang in dataset.langs:
    subs = [(sub_info[sub_uri][lang]['sub'], sub_uri) for sub_uri in sub_info.keys() if lang in sub_info[sub_uri]]
    sub_splitted = chunk_list(subs, 500)
    lang2subs[lang] = sub_splitted
    

In [ ]:
len(lang2subs['en'])

In [ ]:
import subprocess
ARTICLE_ROOT = "/disk/xzhao/datasets/wikipedia_2018_octnov/article"
ARTICLE_GREP_RES_ROOT = "/disk/xzhao/datasets/wikipedia_2018_octnov/article-grep"
TGT_DATA_ROOT = "/disk/xzhao/probing-multilingual/2018_dump_wiki_cache"


def run_grep_match(entities: list[(str, str)], lang, out_fn):
    out_root = os.path.join(TGT_DATA_ROOT, "article_grep_matched", lang)
    os.makedirs(out_root, exist_ok=True)
    out_fn = os.path.join(out_root, out_fn)
    lang_path = os.path.join(ARTICLE_ROOT, lang, '*')
    with open(out_fn, 'w') as fp:
        for entity, entity_uri in entities:
            p = subprocess.run(f'grep -n "{entity}" {lang_path}', shell=True, capture_output=True, text=True)
            lines = p.stdout.split('\n')
            for line in lines:
                newline = f"{entity_uri}:{entity}:{line}"
                fp.write(f"{newline}\n")

def chunk_list(input_list, chunk_size):
    res = []
    for i in range(0, len(input_list), chunk_size):
        res.append(input_list[i:i+chunk_size])
    return res

sub_info = dataset.sub_info
lang2subs = {}
for lang in dataset.langs:
    subs = [(sub_info[sub_uri][lang]['sub'], sub_uri) for sub_uri in sub_info.keys() if lang in sub_info[sub_uri]]
    sub_splitted = chunk_list(subs, 500)
    for idx, sub_list in enumerate(sub_splitted):
        run_grep_match([("蘇丹艾哈邁德清真寺", "Qtest")], "zh", f"{lang}-{idx}.txt")

    

In [ ]:
lines[2]

In [ ]:
import os

for lang in os.listdir("/disk/xzhao")

In [ ]:
import re
prefix = "jawiki-20181120"
pattern = f"{prefix}-pages-articles[0-9]*.xml-.*.bz2"
text = "jawiki-20181120-pages-articles1.xml-p1p106175.bz2"

re.match(pattern, text)

In [ ]:
from transformers import AutoTokenizer, BertTokenizer, BertJapaneseTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
tokenizer.tokenize("日本")

In [ ]:
from mask_dataset import MaskedDataset
from tqdm import tqdm 

dataset = MaskedDataset(model_name="mbert")
WIKI_DUMP_RESULT_ROOT = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache2/subject_object"
os.makedirs(WIKI_DUMP_RESULT_ROOT, exist_ok=True)
sub_info = dataset.get_sub_info()
obj_info = dataset.get_obj_info()

for lang in tqdm(dataset.langs, desc="Writing tokenized subject and object to text file"):
    file_root = os.path.join(WIKI_DUMP_RESULT_ROOT, lang)
    os.makedirs(file_root, exist_ok=True)

    sub_fn = os.path.join(file_root, "subject.text")
    obj_fn = os.path.join(file_root, "object.text")

    with open(sub_fn, 'w') as fp:
        for lang2sub in sub_info.values():
            if lang in lang2sub:
                fp.write(f"{' '.join(lang2sub[lang]['sub_tokens'])}\n")
    with open(obj_fn, 'w') as fp:
        for lang2obj in obj_info.values():
            if lang in lang2obj:
                fp.write(f"{' '.join(lang2obj[lang]['obj_tokens'])}\n")

In [ ]:

from tqdm import tqdm


In [ ]:
import os
import shutil
SUBOBJ_ROOT = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache/tokenized_subject_object"
ABS_ROOT = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache/abstracts_tokenized"
TITLE_ROOT = "/home/xzhao/workspace/probing-mulitlingual/datasets/2018_dump_wiki_cache/titles_tokenized"

langs = ['ms', 'ca', 'ko', 'he', 'fi', 'ga', 'ka', 'en', 'th', 'nl', 
        'zh', 'ja', 'eu', 'da', 'pt', 'ru', 'fr', 'sr', 'et', 'sv', 
        'hy', 'cy', 'sq', 'it', 'hi', 'hr', 'es', 'hu', 'bg', 'ta', 
        'sl', 'bn', 'de', 'id', 'uk', 'be', 'ceb', 'el', 'fa', 'pl', 
        'az', 'ar', 'la', 'gl', 'lt', 'cs', 'sk', 'lv', 'tr', 'af', 
        'vi', 'ur', 'ro']

for lang in langs:
    lang_path = os.path.join(TITLE_ROOT, lang)
    for fn in os.listdir(lang_path):
        tgt_fh = open(os.path.join(lang_path, "all_title.txt"), 'w')
        if "title.txt" in fn:
            fp = os.path.join(lang_path, fn)
            with open(fp, 'r') as fh:
                for line in fh:
                    tgt_fh.write(line.replace('\t', ' ') + '\n')
        tgt_fh.close()

In [ ]:
from tqdm import tqdm
from wiki_2018_dump import xml_doc_iterator
file_path = "/disk/xzhao/datasets/wikipedia_2018_octnov/abstract/en/enwiki-20181120-abstract.xml"
iterator = xml_doc_iterator(file_path)

for i, _ in tqdm(enumerate(iterator)):
    pass



In [ ]:
from wiki_2018_dump import locate_urls, get_title_gz_urls, get_abstract_gz_urls
from tqdm import tqdm 

title_paths = get_title_gz_urls(reload=False)
lang2abstract = get_abstract_gz_urls(reload=False)

In [ ]:
from wikidata.graelo_wiki import load_retraining_text_by_lang
matched = 0
unmatched = 0
for lang in dataset.langs:
    matched_sub, unmatched_sub, _, matched_obj, unmatched_obj, _ = load_retraining_text_by_lang(dataset=dataset, lang='zh')
    matched += matched_sub
    matched += matched_obj
    unmatched += unmatched_sub
    unmatched += unmatched_obj


In [ ]:

wikipedia = load_dataset("graelo/wikipedia", "20230601.zh", split='train', cache_dir="/disk/xzhao/datasets/huggingface")

def search_index(queries, lang, search_dataset=None):
    if search_dataset is None:
        search_dataset = load_dataset("graelo/wikipedia", f"20230601.{lang}", split='train', cache_dir="/disk/xzhao/datasets/huggingface")
    
    search_dataset.load_elasticsearch_index('title', es_client=es_client, es_index_name=f"{lang}_title")
    return search_dataset.get_nearest_examples_batch(index_name="title", queries=queries, k=20)[1]


In [ ]:
wikipedia.load_elasticsearch_index('title', es_client=es_client, es_index_name="zh_title")
_, retrieved_in_title = wikipedia.get_nearest_examples("title", query, k=10)
retrieved_in_title['title']

In [ ]:
wikipedia.load_elasticsearch_index('text', es_client=es_client, es_index_name="zh_text")
_, retrieved_in_text = wikipedia.get_nearest_examples("text", query, k=10)
retrieved_in_text['title']

In [ ]:
from datasets import load_dataset
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

def add_index(es_client, lang):
    wikipedia = load_dataset("graelo/wikipedia", f"20230601.{lang}", split='train', cache_dir="/disk/xzhao/datasets/huggingface")
    if not es_client.indices.exists(index=f"{lang}_title"):
        print(f"Start to index {lang} title")
        wikipedia.add_elasticsearch_index(column="title", es_client=es_client, es_index_name=f"{lang}_title")
    else:
        print(f"Indexing of {lang} title is already done")
    if not es_client.indices.exists(index=f"{lang}_text"):
        print(f"Start to index {lang} text")
        wikipedia.add_elasticsearch_index(column="text", es_client=es_client, es_index_name=f"{lang}_text")
    else:
        print(f"Indexing of {lang} text is already done")
    
def search_index(query, lang):
    wikipedia = load_dataset("graelo/wikipedia", f"20230601.{lang}", split='train')   
    wikipedia.load_elasticsearch_index('title', es_client=es_client, es_index_name=f"{lang}_title")
    wikipedia.load_elasticsearch_index('title', es_client=es_client, es_index_name=f"{lang}_text")
    _, retrieved_in_title = wikipedia.get_nearest_examples("title", query, k=10)
    _, retrieved_in_text = wikipedia.get_nearest_examples("title", query, k=10)
    return retrieved_in_title, retrieved_in_text

# search_index("ルイ・ジュール・トロシュ", 'ja')
add_index(es_client, 'zh')

In [ ]:
query = "ロジェ・ニミエ"
retrieved_examples = search_index(query, 'ja')

In [ ]:
list(retrieved_examples.keys())